#对所写的代码的解释：

这段代码主要是用来对词向量模型进行初始化、训练和评估，以下是各个部分的简要说明：






初始化部分：导入所需的库，并定义了一系列函数和类来实现词向量模型的训练、评估和可视化。

display函数：用于展示数据。

read_json函数：用于从JSON文件中读取数据。

select_model函数：根据模型名称选择对应的模型，如Word2Vec或FastText。






Model类：包含了模型的初始化、预训练、训练和评估方法。

__init__方法：初始化模型参数。

cut_words方法：对文本数据进行分词处理。

pre_train_model方法：预训练一个词向量模型。

random_samples_trainmodel方法：使用随机采样进行模型训练。

epoches_samples_trainmodel方法：将数据分批进行模型训练。

prove_model方法：评估模型性能。






versionism类：包含了对训练好的模型进行可视化和评估的方法。

__init__方法：初始化模型。

demosion_h22方法：使用t-SNE对词向量进行降维并保存。

vector_2d_show方法：展示词向量的二维可视化结果。

cpka方法：展示两个词之间的余弦相似度，并在二维可视化图上标记这两个词及其相似词。

In [ ]:
#初始化
import json
from gensim.models import Word2Vec
import jieba
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import re
import random
from tqdm import tqdm
import time
import os

In [ ]:
# 显示数据
def display(data: list):
    print('len:', len(data))
    for i in range(5):
        print(data[i])

# 读取JSON文件并提取特定字符的数据
def read_json(path_way, character):
    data = []
    with open(path_way, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            if line != '':
                sample = json.loads(line)
                data.append(sample.copy())
    # 从数据中提取指定的字符数据
    train_data = [i[character] for i in data[:]]
    return train_data

# 选择模型
def select_model(model_name):
    if model_name.lower() == 'word2vec':
        return Word2Vec
    elif model_name.lower() == 'fasttext':
        return FastText
    else:
        return None

In [ ]:
class Model():
    def __init__(self, model_name, args, model_path):#设置参数，数据，模型信息
        """
        初始化Model对象。

        参数：
            - model_name (str): 模型名称，支持'word2vec'或'fasttext'。
            - args (dict): 模型参数。
            - model_path (str): 模型保存路径。

        """
        self.paras = args
        self.model_path = model_path
        self.model = select_model(model_name)  # 根据模型名称选择模型
        
    def cut_words(self, train_data):
        """
        对文本进行分词处理。

        参数：
            - train_data (list): 待分词的文本数据。

        返回：
            - seg_data (list): 分词后的文本数据。

        """
        seg_data = list()
        for i in train_data:
            new_text = "".join(re.findall('[\u4e00-\u9fa5]+', i, re.S))  # 去除一些无用的字符只提取出中文出来
            seg_i = jieba.lcut(new_text)
            seg_data.append(seg_i)
        return seg_data
    
    def pre_train_model(self, data):
        """
        预训练一个模型。

        参数：
            - data (list): 训练数据。

        """
        seg_data = self.cut_words(data[:1000])  # 只使用前1000条数据进行预训练
        Model = self.model(sentences=seg_data, **self.paras)  # 根据参数初始化模型
        Model.save(self.model_path)  # 保存模型
    
    def random_samples_trainmodel(self, data, sample_size, epoches):
        """
        使用随机采样进行模型训练。

        参数：
            - data (list): 训练数据。
            - sample_size (int): 每次采样的样本数量。
            - epoches (int): 训练轮数。

        """
        if not os.path.exists(self.model_path):
            self.pre_train_model(data)  # 如果模型不存在，则预训练一个模型
        Model = self.model.load(self.model_path)  # 加载预训练的模型
        with tqdm(total=epoches, desc="Training", unit="step") as pbar:
            for _ in range(epoches):
                random_samples = random.sample(data, sample_size)  # 随机采样
                seg_data = self.cut_words(random_samples)  # 分词处理
                Model.train(seg_data, total_examples=len(seg_data), epochs=1)  # 训练模型
                pbar.update(1)  # 更新进度条
            Model.save(self.model_path)  # 保存模型
    
    def epoches_samples_trainmodel(self, data, delt_data, size):
        """
        使用分批数据进行模型训练。

        参数：
            - data (list): 训练数据。
            - delt_data (int): 每个批次的数据量。
            - size (int): 总批次数。

        """
        if not os.path.exists(self.model_path):
            self.pre_train_model(data)  # 如果模型不存在，则预训练一个模型
        Model = self.model.load(self.model_path)  # 加载预训练的模型
        with tqdm(total=size, desc="Training", unit="step") as pbar:
            for t in range(1, size + 1):
                cut_data = data[delt_data * (t - 1):delt_data * t]  # 切割数据
                seg_data = self.cut_words(cut_data)  # 分词处理
                Model.train(seg_data, total_examples=len(seg_data), epochs=1)  # 训练模型
                pbar.update(1)  # 更新进度条
            Model.save(self.model_path)  # 保存模型
        
    def prove_model(self):
        """
        评估模型性能。

        """
        model = self.model.load(self.model_path)  # 加载模型
        print("查看词向量大小:", model.wv.vectors.shape)  # 查看词向量维度
        print("计算两个词的余弦相似度:", model.wv.similarity('中国', '美国'))  # 计算两个词的余弦相似度
        print("取出与“中国”最相似的10个词", model.wv.most_similar('中国', topn=10))  # 获取与指定词最相似的词汇
        print("获得 国王-男人+女人 的词，理应为女王，而实际上最接近的10个词为：",
              model.wv.most_similar(positive=["男人", "国王"], negative=["女人"], topn=10))  # 进行类比测试


In [ ]:
class versionism():
    def __init__(self, model_name, model_path, save_path):
        """
        初始化versionism对象。

        参数：
            - model_name (str): 模型名称，支持'word2vec'或'fasttext'。
            - model_path (str): 预训练模型的路径。
            - save_path (str): 保存降维后词向量的路径。

        """
        self.save_path = save_path
        modeltype = select_model(model_name)
        self.model = modeltype.load(model_path)
        
    def demosion_h22(self):
        """
        使用t-SNE算法将词向量降维至二维，并保存降维后的结果。

        """
        model = self.model
        words = list(model.wv.key_to_index)  # 获取模型中所有词语的词向量
        vectors = model.wv[words]  # 获取词向量
        tsne = TSNE(n_components=2, random_state=42)  # 创建t-SNE对象
        vectors_2d = tsne.fit_transform(vectors)  # 将词向量降维至二维
        np.save(self.save_path, vectors_2d)  # 保存降维后的结果
        print("Finished saving vector_2d")
        
    def vector_2d_show(self):
        """
        将降维后的词向量进行可视化展示。

        """
        if not os.path.exists(self.save_path):
            self.demosion_h22()  # 如果降维后的词向量不存在，则进行降维处理
         
        vectors_2d = np.load(self.save_path)  # 加载降维后的词向量
        model = self.model
        words = list(model.wv.key_to_index)  # 获取模型中所有词语的词向量

        plt.rcParams['font.sans-serif'] = ['SimHei']
        plt.rcParams['axes.unicode_minus'] = False

        plt.figure(figsize=(10, 8))  # 创建一个二维可视化图
        plt.scatter(vectors_2d[:1000, 0], vectors_2d[:1000, 1], marker='o', s=8)  # 绘制散点图
        for i, word in enumerate(words[:1000]):
            plt.annotate(word, xy=(vectors_2d[i, 0], vectors_2d[i, 1]), fontsize=20)  # 添加词语标签

        plt.title('t-SNE Visualization of Word2Vec Embeddings')  # 添加图标题
        plt.xlabel('t-SNE Dimension 1')  # 添加x轴标签
        plt.ylabel('t-SNE Dimension 2')  # 添加y轴标签

        plt.show()  # 显示可视化图
        
    def cpka(self, word1, word2):
        """
        将两个词汇在二维空间中的位置进行可视化展示，并计算它们的余弦相似度。

        参数：
            - word1 (str): 第一个词汇。
            - word2 (str): 第二个词汇。

        """
        if not os.path.exists(self.save_path):
            self.demosion_h22()  # 如果降维后的词向量不存在，则进行降维处理
        vectors_2d = np.load(self.save_path)  # 加载降维后的词向量 
        
        model = self.model
        words = list(model.wv.key_to_index)  # 获取模型中所有词语的词向量

        china = [words.index(word) for word, _ in model.wv.most_similar(positive=[word1], topn=100)]  # 取出与“中国”最相似的100个词
        america = [words.index(word) for word, _ in model.wv.most_similar(positive=[word2], topn=100)]  # 取出与“美国”最相似的100个词

        plt.figure(figsize=(10, 8))  # 创建一个二维可视化图
        plt.scatter(vectors_2d[china, 0], vectors_2d[china, 1], marker='o', s=8)  # 绘制散点图
        plt.scatter(vectors_2d[america, 0], vectors_2d[america, 1], marker='o', s=8)  # 绘制散点图
        for i in china:
            plt.annotate(words[i], xy=(vectors_2d[i, 0], vectors_2d[i, 1]), fontsize=20, color='red')  # 添加词语标签
        for i in america:
            plt.annotate(words[i], xy=(vectors_2d[i, 0], vectors_2d[i, 1]), fontsize=20, color='blue')  # 添加词语标签
        plt.show()  # 显示可视化图

        print("两个词的余弦相似度:", model.wv.similarity(word1, word2))  # 计算两个词的余弦相似度
